In [72]:
import pandas as pd
import os

import itertools as it

from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

In [25]:
ing_df_orig = pd.read_excel('data/ingredients.xlsx')
color_df_orig= pd.read_excel('data/color_results.xlsx')

In [26]:
print(ing_df_orig.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Date Issued                 66 non-null     datetime64[ns]
 1   Plan Month                  22 non-null     object        
 2   Material                    230 non-null    object        
 3   Order                       66 non-null     float64       
 4   RMIX Batch                  230 non-null    int64         
 5   Antimony Oxide              230 non-null    object        
 6   Manganic Oxide              230 non-null    object        
 7   Titanium Dioxide - Anatase  230 non-null    object        
 8   Kiln                        66 non-null     float64       
 9   Hearth                      0 non-null      float64       
 10  Cycle                       0 non-null      float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(5)
memo

In [27]:
ing_df_wk =  ing_df_orig.drop(columns=['Date Issued', 'Plan Month', 'Order',
       'Kiln', 'Hearth', 'Cycle'])

# ing_df_wk.drop(['Date Issued', 'Plan Month', 'Order','Kiln', 'Hearth', 'Cycle'], axis=1, inplace=True)

In [28]:
# ing_df_orig[['RMIX Batch', 'Kiln', 'Hearth','Cycle']] = ing_df_orig[['RMIX Batch', 'Kiln', 'Hearth','Cycle']].astype(str)

In [41]:
ing_df_wk['RMIX Batch'] = ing_df_wk['RMIX Batch'].astype(str)

In [42]:
ing_df_wk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Material                    230 non-null    object
 1   RMIX Batch                  230 non-null    object
 2   Antimony Oxide              230 non-null    object
 3   Manganic Oxide              230 non-null    object
 4   Titanium Dioxide - Anatase  230 non-null    object
dtypes: object(5)
memory usage: 9.1+ KB


In [31]:
# print(color_df_orig.info())

In [32]:
print(color_df_orig.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   RMIX Batch  221 non-null    int64  
 1   Fin Batch   221 non-null    int64  
 2   Equip       221 non-null    object 
 3   MDL         221 non-null    float64
 4   MDA         221 non-null    float64
 5   MDB         221 non-null    float64
 6   TDL         221 non-null    float64
 7   TDA         221 non-null    float64
 8   TDB         221 non-null    float64
 9   STR         221 non-null    int64  
dtypes: float64(6), int64(3), object(1)
memory usage: 17.4+ KB
None


In [33]:
color_df_orig[['RMIX Batch', 'Fin Batch']] = color_df_orig[['RMIX Batch', 'Fin Batch']].astype(str)

In [34]:
color_df_wk = color_df_orig

In [35]:
color_df_wk

,RMIX Batch,Fin Batch,Equip,MDL,MDA,MDB,TDL,TDA,TDB,STR
0,23633,23799,MX003,-1.0,-0.4,-0.9,-0.7,-0.4,-1.0,104
1,35324,35816,MX003,-0.2,-0.2,-0.5,-0.3,-0.4,-0.8,102
2,42908,53780,MX007,0.0,-0.1,-0.2,0.2,-0.1,-0.2,98
3,42909,53781,MX003,0.2,-0.2,-0.4,-0.1,-0.1,-0.3,100
4,42910,53782,MX003,0.4,0.0,-0.2,0.2,0.0,0.0,98
...,...,...,...,...,...,...,...,...,...,...
216,40693,40705,MX003,0.4,-0.1,-0.2,0.5,-0.2,-0.1,97
217,40694,40706,MX003,0.1,-0.2,-0.2,0.6,-0.3,-0.6,95
218,41775,41792,MX003,-0.3,0.0,-0.1,-0.1,-0.1,-0.2,100
219,41779,41794,MX003,0.1,0.0,0.1,-0.1,0.0,0.2,101


In [43]:
df = ing_df_wk.merge(color_df_wk,how='left',left_on='RMIX Batch',right_on='RMIX Batch')

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 231 entries, 0 to 230
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Material                    231 non-null    object 
 1   RMIX Batch                  231 non-null    object 
 2   Antimony Oxide              231 non-null    object 
 3   Manganic Oxide              231 non-null    object 
 4   Titanium Dioxide - Anatase  231 non-null    object 
 5   Fin Batch                   203 non-null    object 
 6   Equip                       203 non-null    object 
 7   MDL                         203 non-null    float64
 8   MDA                         203 non-null    float64
 9   MDB                         203 non-null    float64
 10  TDL                         203 non-null    float64
 11  TDA                         203 non-null    float64
 12  TDB                         203 non-null    float64
 13  STR                         203 non

In [45]:
df = df.dropna()

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 203 entries, 1 to 220
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Material                    203 non-null    object 
 1   RMIX Batch                  203 non-null    object 
 2   Antimony Oxide              203 non-null    object 
 3   Manganic Oxide              203 non-null    object 
 4   Titanium Dioxide - Anatase  203 non-null    object 
 5   Fin Batch                   203 non-null    object 
 6   Equip                       203 non-null    object 
 7   MDL                         203 non-null    float64
 8   MDA                         203 non-null    float64
 9   MDB                         203 non-null    float64
 10  TDL                         203 non-null    float64
 11  TDA                         203 non-null    float64
 12  TDB                         203 non-null    float64
 13  STR                         203 non

In [70]:
df['Material'].dtype == 'object'

True

In [62]:
for column in df.columns:
    print(f'{column} (' + str(df.loc[df[column]==''].count().iloc[0]) + ' empty strings)')

Material (0 empty strings)
RMIX Batch (0 empty strings)
Antimony Oxide (0 empty strings)
Manganic Oxide (0 empty strings)
Titanium Dioxide - Anatase (0 empty strings)
Fin Batch (0 empty strings)
Equip (0 empty strings)
MDL (0 empty strings)
MDA (0 empty strings)
MDB (0 empty strings)
TDL (0 empty strings)
TDA (0 empty strings)
TDB (0 empty strings)
STR (0 empty strings)


In [89]:
approv_ingred_df = pd.read_excel('data/approved_ingredients.xlsx')
formula_ingred_df = pd.read_excel('data/formulas.xlsx')

product_ingredients = formula_ingred_df['Ingredient'].tolist()
approved_list = approv_ingred_df['rm_code'].tolist()
print(approved_list)
print(product_ingredients)

listA = ['a','b','c','d','e']
listB = ['c','d','e']
listC = ['f','g','h']

listA = (list(it.combinations(['Antimony Oxide', 'Manganic Oxide', 'Titanium Dioxide - Anatase'],3)))
listB = (list(it.combinations(['Antimony Oxide', 'Antimony Oxide', 'Titanium Dioxide - Anatase'],3)))
listC = (list(it.combinations(['Antimony Oxide', 'Manganic Oxide', 'Manganic Oxide'],3)))

print()

# xy = (zip(listA,listB))
# print(list(xy))
# # new_list = 


# convert lists to sets and test equality before adding to a unique list
# if set(product_ingredients) == for each comb in list(it.combinations(approved_list,3))





['RM0615', 'RM0614', 'RM4620', 'RM4621', 'RM4615', 'RM4623', 'RM4626', 'RM6729', 'RM6707', 'RM6718', 'RM6713', 'RM6714']
['Antimony Oxide', 'Manganic Oxide', 'Titanium Dioxide - Anatase']
[('Antimony Oxide', 'Manganic Oxide', 'Titanium Dioxide - Anatase')]
[('Antimony Oxide', 'Antimony Oxide', 'Titanium Dioxide - Anatase')]
[('Antimony Oxide', 'Manganic Oxide', 'Manganic Oxide')]
